# Imports

In [41]:
import os
import sys
import json
import random
from datetime import datetime

import pandas as pd
import networkx as nx
from shapely.geometry import shape, MultiPolygon
from shapely.ops import unary_union

In [42]:
# Obtener ruta absoluta del directorio que contiene el notebook
notebook_dir = os.path.dirname(os.getcwd())  # sube un nivel desde /notebook
if notebook_dir not in sys.path:
    sys.path.append(notebook_dir)

In [43]:
# Funciones de tu proyecto
from generate_area.select_area import get_area_bounds
from generate_area.wind_data_meh import batch_evaluate_wind
from generate_area.nautical_constraints_meh import (
    load_forbidden_zones,
    mark_blocked_seamark,
    fetch_seamark_elements,
    fetch_coastline_elements,
    classify_seamarks,
    create_buoy_nodes,
    combine_mesh_and_buoys,
    compute_navigable_final
)
from generate_area.build_graph import build_weighted_graph, PolarDiagram
from pathfinding.search import shortest_path_with_turn_penalty

# Parametros

In [44]:
# 1) Parámetros de área
path_geojson = '../../data/raw/selection_1.geojson'
lat_min, lat_max, lon_min, lon_max = get_area_bounds(path_geojson)

In [45]:
# 2) Directorio de salida
out_dir = '../../data/expert_trajectories'
os.makedirs(out_dir, exist_ok=True)

graph_dir = '../../data/graphs'
os.makedirs(graph_dir, exist_ok=True)

In [46]:
# 3) Parámetros de generación
n_dates         = 25
routes_per_date = 20
min_route_len   = 15       # mínimo de nodos por ruta
start_str       = '2025-05-04'
end_str         = '2025-06-19'
boat_data       = {'max_wind_speed': 25.0}  # nudos
beta_turn       = 0.1                       # penalización de virada

In [47]:
# 4) Cargar datos estáticos
bathy_df    = pd.read_csv('../../data/raw/malla/nodes_bathy.csv')
polar_df    = pd.read_csv('../../data/raw/malla/polar_diagram.csv')
zones_fc    = json.load(open('../../data/raw/malla/non_navigable_zones.json', 'r'))
polys       = [shape(feat['geometry']) for feat in zones_fc['features']]
union_restr = unary_union(polys)

In [48]:
# 5) Inicializar diagrama polar
polar = PolarDiagram(polar_df)

# 6) Fechas equiespaciadas
dates = pd.date_range(start=start_str, end=end_str, periods=n_dates).to_pydatetime().tolist()

# test para ver funcionalidad
dates = dates[:1] 


# Generación de rutas

In [ ]:
for date in dates:
    date_str = date.strftime('%Y-%m-%d')
    print(f"[{date_str}] Generando {routes_per_date} rutas...")

    # 7.1) Obtener viento para la fecha
    wind_res = batch_evaluate_wind(
        bathy_df.to_dict('records'),
        boat_data,
        date_str,
        date_str
    )
    df_wind = pd.DataFrame(wind_res)

    # 7.2) Unir batimetría + viento
    nodes = bathy_df.merge(
        df_wind[['latitude','longitude','time','wind_speed_10m','wind_direction_10m']],
        on=['latitude','longitude'],
        how='inner'
    )

    nodes.to_csv('../../data/graphs/nodes_bathy_wind.csv', index=False)
    
    graph_path = os.path.join(graph_dir, f"nodes_bathy_wind{date.strftime('%Y%m%d')}.csv")
    nodes.to_csv( graph_path)
    print(f"  → Nodos de profundidad y viento guardado en {graph_path}")
    
    # 7.3) Cargar zonas prohibidas (seamarks + línea de costa)
    tree = load_forbidden_zones(lat_min, lon_min, lat_max, lon_max, seguridad=0.01)
    restricted_geoms = list(tree.geometries)

    # 7.4) Marcar los nodos bloqueados por seamarks/costa
    nodes['blocked_seamark'] = mark_blocked_seamark(nodes, restricted_geoms, seguridad=0.01)

    # 7.5) Obstáculos y boyas
    elements   = fetch_seamark_elements(lat_min, lon_min, lat_max, lon_max)
    buoy_geoms = classify_seamarks(elements)
    buoy_df    = create_buoy_nodes(buoy_geoms)

    # 7.6) Unificar datos y navegabilidad final
    combined = combine_mesh_and_buoys(nodes, buoy_df)
    combined['navigable_final'] = compute_navigable_final(combined)

    with open('../../data/raw/malla/non_navigable_zones.json', 'r', encoding='utf-8') as f:
        zones_fc = json.load(f)
    print("Zonas no navegables realizadas")
        
    # 7.7) Construir la geometría unida de zonas no navegables
    geoms = [shape(feat['geometry']) for feat in zones_fc['features']]
    if len(geoms) > 1:
        union_restr = MultiPolygon(geoms)
    else:
        union_restr = geoms[0]

    print("Construccion de grafo")
    
    # 7.8) Escoge un sólo time‐slice (la primera hora disponible)
    hora0 = combined['time'].unique()[0]
    static_df = combined[combined['time'] == hora0].copy()

    # 7.9) O bien, simplemente elimina duplicados lat-lon, KEEP=first
    static_df = static_df.drop_duplicates(subset=['latitude','longitude'], keep='first')

    # Opcional: comprueba cuántos nodos únicos tienes ahora
    total_nodos = len(static_df)
    print(f"Nodos únicos para grafo: {total_nodos}")
    
    # 7.10) Construir grafo ponderado con turn-penalty
    G = build_weighted_graph(
        static_df,
        polar_df,
        union_restr,
        max_neighbors=32,
        neighbor_cells=3,
        alpha_time=1.0,
        beta_comfort=0.1,
        beta_turn=beta_turn
    )

    graph_path = os.path.join(graph_dir, f"graph_{date.strftime('%Y%m%d')}.gpickle")
    nx.write_gpickle(G, graph_path)
    print(f"  → Grafo guardado en {graph_path}")

    # 7.11) Selección de rutas con longitud mínima
    node_ids = list(G.nodes())
    if len(node_ids) < 2:
        print(f"  ⚠️ Sólo {len(node_ids)} nodos navegables; omitiendo {date_str}.")

    generated = 0
    attempts  = 0
    max_attempts = routes_per_date * 10

    print("Construccion de rutas")

    while generated < routes_per_date and attempts < max_attempts:
        attempts += 1
        src, tgt = random.sample(node_ids, 2)
        result = shortest_path_with_turn_penalty(G, src, tgt, beta_turn=beta_turn)
        path = result['path']
        # Verificar que exista ruta y cumpla la longitud mínima
        if not path or len(path) < min_route_len:
            continue

        # Construir y guardar trayectoria por tramos
        traj = []
        prev_heading = None
        for step, (u, v) in enumerate(zip(path[:-1], path[1:])):
            d_u        = G.nodes[u]
            lat_u      = d_u['latitude']
            lon_u      = d_u['longitude']
            wind_dir_u = d_u['wind_dir']
            wind_spd_u = d_u['wind_speed']

            edge_attrs   = G[u][v]
            heading_uv   = edge_attrs['heading']
            twa          = abs(((heading_uv - wind_dir_u) + 180) % 360 - 180)
            speed_uv     = polar.get_speed(twa)
            heading_actual = prev_heading if prev_heading is not None else heading_uv

            traj.append({
                'date':           date_str,
                'route_id':       f"{date.strftime('%Y%m%d')}_{generated:02d}",
                'step':           step,
                'latitude':       lat_u,
                'longitude':      lon_u,
                'heading':        heading_actual,
                'speed':          speed_uv,
                'wind_dir':       wind_dir_u,
                'wind_speed':     wind_spd_u,
                'target_heading': heading_uv,
                'target_speed':   speed_uv
            })
            prev_heading = heading_uv

        out_path = os.path.join(out_dir, f"{date.strftime('%Y%m%d')}_route_{generated:02d}.csv")
        pd.DataFrame(traj).to_csv(out_path, index=False)

        generated += 1

    print(f"  ✔️ Generadas {generated}/{routes_per_date} rutas en {out_dir}")

print("✅ Generación de rutas completada.")


# Visualización de las rutas

In [40]:
import os
import glob

import pandas as pd
import folium
import matplotlib.pyplot as plt
from matplotlib import cm, colors

# 1) Directorio donde guardas tus CSV de rutas
routes_dir = '../../data/expert_trajectories'

# 2) Leer todos los CSV y concatenarlos
all_files = glob.glob(os.path.join(routes_dir, '*.csv'))
#all_files = all_files[:1] #-> para pirar solo la primera ruta
df_list = []
for f in all_files:
    df = pd.read_csv(f)
    df_list.append(df)
routes_df = pd.concat(df_list, ignore_index=True)

# 3) Obtener centro del mapa (media de todas las coordenadas)
center_lat = routes_df['latitude'].mean()
center_lon = routes_df['longitude'].mean()

# 4) Crear mapa base
m = folium.Map(location=[center_lat, center_lon], zoom_start=8)

# 5) Crear paleta de colores para cada ruta única
route_ids = routes_df['route_id'].unique()
n_routes = len(route_ids)
colormap = cm.get_cmap('tab20', n_routes)
route_color = {rid: colors.rgb2hex(colormap(i)) for i, rid in enumerate(route_ids)}

# 6) Añadir cada ruta al mapa
for rid, group in routes_df.groupby('route_id'):
    coords = list(zip(group['latitude'], group['longitude']))
    folium.PolyLine(
        locations=coords,
        color=route_color[rid],
        weight=2,
        opacity=0.8,
        tooltip=rid
    ).add_to(m)
    # opcional: marcador inicio y fin
    folium.CircleMarker(location=coords[0], radius=4, color=route_color[rid],
                        fill=True, fill_opacity=1.0,
                        popup=f"{rid} start").add_to(m)
    folium.CircleMarker(location=coords[-1], radius=4, color=route_color[rid],
                        fill=True, fill_opacity=1.0,
                        popup=f"{rid} end").add_to(m)

# 7) Mostrar el mapa
m


C:\Users\aigut\AppData\Local\Temp\ipykernel_46388\301653263.py:31: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colormap = cm.get_cmap('tab20', n_routes)
